In [1]:
%run ../utils.ipynb

[2024-01-15 14:07:12,613 INFO] [numexpr.utils.utils._init_num_threads] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
import os
import sys

from dotenv import load_dotenv, find_dotenv

from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2024-01-15 14:07:30,506 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2024-01-15 14:07:30,510 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2024-01-15 14:07:30,512 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2024-01-15 14:07:30,514 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2024-01-15 14:07:30,578 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2024-01-15 14:07:30,582 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2024-01-15 14:07:30,582 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


In [3]:
auth_dir = f"{project_dir}/auth"

In [4]:
Game_Keys

,season,code
16,2015,348
17,2016,359
18,2017,371
19,2018,380
20,2019,390
21,2020,399
22,2021,406
23,2022,414
24,2023,423


In [5]:
League_Ids

{'2015': 853925,
 '2016': 810238,
 '2017': 192551,
 '2018': 799346,
 '2019': 820083,
 '2020': 394682,
 '2021': 364325,
 '2022': 564704,
 '2023': 333146}

In [6]:
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id= 333146,
    game_code='nfl',
    game_id= 423
)

In [27]:
teams = yahoo_query.get_league_teams()
team_ids_list = []
for team in teams:
    team_name = str(team.name)[1:].strip("'").strip('"')
    manager = team.managers[0].nickname
    id = team.team_id
    info = {'team_name': team_name, 'manager_nickname': manager, 'id': id}
    team_ids_list.append(info)

#stash team data in dataframe
team_ids_df = pd.DataFrame(team_ids_list)

In [30]:
team_ids_df

,team_name,manager_nickname,id
0,The Ice Cold Bruschi,Ricky,1
1,Immortal Duffmen,Pat,2
2,Jadeveon's Hit Men,Tim,3
3,John Brown's Army,Eric,4
4,Saquon These Nuts,Frags,5
5,Starr is Born,Nick,6
6,SuperCam That Ho,Lauren,7
7,The fishingtons,Buddy,8
8,Gibbs Me A Break,Kyle,9
9,The Twelve Apostles,Jones,10


In [31]:
Query_Auth_Ids

{'2015': [853925, 348],
 '2016': [810238, 359],
 '2017': [192551, 371],
 '2018': [799346, 380],
 '2019': [820083, 390],
 '2020': [394682, 399],
 '2021': [364325, 406],
 '2022': [564704, 414],
 '2023': [333146, 423]}

In [33]:
df = pd.DataFrame()
for year in Query_Auth_Ids.keys():
    yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id= Query_Auth_Ids[year][0],
    game_code='nfl',
    game_id= Query_Auth_Ids[year][1]
    )
    print(f"Query made for season {year}")
    teams = yahoo_query.get_league_teams()
    team_ids_list = []
    for team in teams:
        team_name = str(team.name)[1:].strip("'").strip('"')
        manager = team.managers[0].nickname
        id = team.team_id
        info = {'team_name': team_name, 'manager_nickname': manager, 'id': id}
        team_ids_list.append(info)
    
    #stash team data in dataframe
    team_ids_df = pd.DataFrame(team_ids_list)
    team_ids_df['manager_nickname'] = team_ids_df['team_name'].apply(map_teams_to_name)
    print("Generating player data for:")
    for name, id in zip(team_ids_df['manager_nickname'], team_ids_df['id']):
        try:
            roster = yahoo_query.get_team_roster_player_stats(id)
        except:
            print(f"Error encountered generating roster for manager: {name} in season {year}")
            break
        else:
            print(f"{name}", end="|")
            for player in roster:
                row = pd.json_normalize(player.serialized())
                row['season'] = year
                row['ownership'] = name
                df = pd.concat([df, row])

Query made for season 2015
Generating player data for:
Ricky|Peter|Tim|Big Rick|Michael|Lauren|Jake|Eric|Kyle|Buddy|Pat|Nick|Frags|Jones|Query made for season 2016
Generating player data for:
Ricky|Pat|Tim|Eric|Jones|Lauren|Nick|Buddy|Kyle|Michael|Jake|Frags|Peter|Nate|Query made for season 2017
Generating player data for:
Ricky|Pat|Tim|Eric|Jones|Lauren|Nick|Buddy|Kyle|Michael|Jake|Frags|Peter|Nate|Query made for season 2018
Generating player data for:
Ricky|Pat|Tim|Eric|Jones|Lauren|Nick|Buddy|Kyle|Michael|Jake|Frags|Peter|Nate|Query made for season 2019
Generating player data for:
Ricky|Pat|Tim|Eric|Frags|Nick|Lauren|Buddy|Kyle|Jones|Michael|Jake|Nate|Peter|Query made for season 2020
Generating player data for:
Ricky|Pat|Tim|Eric|Frags|Nick|Lauren|Buddy|Kyle|Jones|Michael|Jake|Nate|Peter|Query made for season 2021
Generating player data for:
Ricky|Pat|Tim|Eric|Frags|Nick|Lauren|Buddy|Kyle|Jones|Michael|Jake|Nate|Peter|Query made for season 2022
Generating player data for:
Ricky|Pat|

In [36]:
df.to_csv(project_dir + 'data\\rosters\\season_rosters_raw.csv')

In [85]:
player_meta = pd.read_csv(project_dir + 'data\\rosters\\season_rosters_raw.csv')

In [86]:
player_meta.loc[player_meta['name.full'].str.contains('Allen Robinson II'), 'display_position'].max()

'WR'

In [87]:
df['W/R/T position'] = df['W/R/T'].apply(lambda x: player_meta.loc[player_meta['name.full'].str.contains(x), 'display_position'].max())

In [88]:
df[['W/R/T', 'W/R/T position']]

,W/R/T,W/R/T position
season_week,,
Ricky|2015|1,Allen Robinson II,WR
Ricky|2015|2,Tre Mason,NaN
Ricky|2015|3,Alfred Morris,RB
Ricky|2015|4,Allen Robinson II,WR
Ricky|2015|5,Carlos Hyde,RB
...,...,...
Peter|2023|12,D'Andre Swift,RB
Peter|2023|13,Garrett Wilson,WR
Peter|2023|14,Saquon Barkley,RB


In [89]:
df.groupby('W/R/T position').count()

,week,team_name,in_season_manager_id,manager_name,opponent_team_name,in_season_opponent_manager_id,opponent_name,projected_points_for,points_for,projected_points_against,...,BN7,BN7_player_key,BN7_points,WR3,WR3_player_key,WR3_points,IR,IR_player_key,IR_points,playoff_week
W/R/T position,,,,,,,,,,,,,,,,,,,,,
RB,628,628,628,628,628,628,628,628,628,628,...,628,628,628,628,628,628,628,628,628,628
"RB,TE",3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
TE,129,129,129,129,129,129,129,129,129,129,...,129,129,129,129,129,129,129,129,129,129
WR,1094,1094,1094,1094,1094,1094,1094,1094,1094,1094,...,1094,1094,1094,1094,1094,1094,1094,1094,1094,1094
"WR,RB",5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [90]:
cols = points_columns


In [91]:
cols.append('W/R/T position')
cols.append('manager')
cols.append('win')
cols.append('points_for')

In [92]:
mwp_df = df.loc[:, cols]

In [93]:
mwp_df['W/R/T position'].fillna('WR', inplace=True)

In [94]:
mwp_df['W/R/T position'].unique()

array(['WR', 'RB', 'TE', 'WR,RB', 'RB,TE'], dtype=object)

In [95]:
mwp_df['W/R/T position'] = mwp_df['W/R/T position'].str.replace('WR,RB', 'WR')

In [96]:
mwp_df['W/R/T position'] = mwp_df['W/R/T position'].str.replace('RB,TE', 'TE')

In [97]:
mwp_df['W/R/T position'].unique()

array(['WR', 'RB', 'TE'], dtype=object)

In [98]:
mwp_df

,QB_points,WR1_points,WR2_points,WR3_points,RB1_points,RB2_points,TE_points,W/R/T_points,DEF_points,BN1_points,BN2_points,BN3_points,BN4_points,BN5_points,BN6_points,BN7_points,W/R/T position,manager,win,points_for
season_week,,,,,,,,,,,,,,,,,,,,
Ricky|2015|1,25.70,7.3,8.8,0.0,31.2,4.3,11.8,3.2,18.5,12.10,7.23,10.4,9.2,17.2,0.0,0.0,WR,Ricky,1,123.80
Ricky|2015|2,25.13,12.0,30.5,0.0,8.1,8.2,0.0,3.0,15.0,0.00,17.03,13.0,0.0,0.0,0.0,0.0,WR,Ricky,1,106.43
Ricky|2015|3,20.33,39.7,8.8,0.0,3.6,6.6,10.3,1.9,5.0,39.80,6.10,0.0,17.3,0.0,0.0,0.0,RB,Ricky,1,101.23
Ricky|2015|4,15.87,11.7,5.2,0.0,2.7,35.4,18.8,10.0,11.0,14.00,0.00,0.0,7.5,0.0,0.0,0.0,WR,Ricky,1,113.67
Ricky|2015|5,22.47,10.8,22.7,0.0,28.2,1.5,5.6,16.2,9.5,3.10,0.00,15.7,0.0,0.0,0.0,0.0,RB,Ricky,1,122.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Peter|2023|12,17.30,15.6,14.7,0.0,5.7,9.9,13.5,8.9,9.0,13.90,11.80,17.9,1.2,6.5,0.0,0.0,RB,Peter,1,101.60
Peter|2023|13,18.18,8.3,0.0,0.0,12.7,3.0,0.0,6.5,6.0,0.00,0.00,0.0,24.1,10.8,0.0,0.0,WR,Peter,0,69.68
Peter|2023|14,23.70,9.8,15.6,0.0,10.7,3.9,7.8,21.6,15.0,0.00,0.00,0.0,13.8,0.0,0.0,0.0,RB,Peter,1,126.10


In [99]:
mwp_df['WR_total_points'] = df[['WR1_points','WR2_points', 'WR3_points']].sum(axis=1) + df['W/R/T_points'] * (df['W/R/T position'] == 'WR')

In [100]:
mwp_df['RB_total_points'] = df[['RB1_points','RB2_points']].sum(axis=1) + df['W/R/T_points'] * (df['W/R/T position'] == 'RB')

In [101]:
mwp_df['TE_total_points'] = df['TE_points'] + df['W/R/T_points'] * (df['W/R/T position'] == 'TE')

In [102]:
points_columns[-11:-4]

['BN1_points',
 'BN2_points',
 'BN3_points',
 'BN4_points',
 'BN5_points',
 'BN6_points',
 'BN7_points']

In [103]:
bench_points = 0
for i in points_columns[-11:-4]:
    bench_points+=mwp_df[i]
mwp_df['BN_points'] = bench_points

In [104]:
mwp_df.columns

Index(['QB_points', 'WR1_points', 'WR2_points', 'WR3_points', 'RB1_points',
       'RB2_points', 'TE_points', 'W/R/T_points', 'DEF_points', 'BN1_points',
       'BN2_points', 'BN3_points', 'BN4_points', 'BN5_points', 'BN6_points',
       'BN7_points', 'W/R/T position', 'manager', 'win', 'points_for',
       'WR_total_points', 'RB_total_points', 'TE_total_points', 'BN_points'],
      dtype='object')

In [105]:
rearranged_cols = ['QB_points', 'WR1_points', 'WR2_points', 'WR3_points', 'RB1_points',
       'RB2_points', 'TE_points', 'W/R/T_points', 'DEF_points', 'BN1_points',
       'BN2_points', 'BN3_points', 'BN4_points', 'BN5_points', 'BN6_points',
       'BN7_points', 'W/R/T position', 
       'WR_total_points', 'RB_total_points', 'TE_total_points', 'BN_points', 'manager', 'win', 'points_for']

In [106]:
mwp_df = mwp_df[rearranged_cols]

In [107]:
mwp_df.head()

,QB_points,WR1_points,WR2_points,WR3_points,RB1_points,RB2_points,TE_points,W/R/T_points,DEF_points,BN1_points,...,BN6_points,BN7_points,W/R/T position,WR_total_points,RB_total_points,TE_total_points,BN_points,manager,win,points_for
season_week,,,,,,,,,,,,,,,,,,,,,
Ricky|2015|1,25.70,7.3,8.8,0.0,31.2,4.3,11.8,3.2,18.5,12.1,...,0.0,0.0,WR,19.3,35.5,11.8,56.13,Ricky,1,123.80
Ricky|2015|2,25.13,12.0,30.5,0.0,8.1,8.2,0.0,3.0,15.0,0.0,...,0.0,0.0,WR,42.5,16.3,0.0,30.03,Ricky,1,106.43
Ricky|2015|3,20.33,39.7,8.8,0.0,3.6,6.6,10.3,1.9,5.0,39.8,...,0.0,0.0,RB,48.5,12.1,10.3,63.20,Ricky,1,101.23
Ricky|2015|4,15.87,11.7,5.2,0.0,2.7,35.4,18.8,10.0,11.0,14.0,...,0.0,0.0,WR,26.9,38.1,18.8,21.50,Ricky,1,113.67
Ricky|2015|5,22.47,10.8,22.7,0.0,28.2,1.5,5.6,16.2,9.5,3.1,...,0.0,0.0,RB,33.5,45.9,5.6,18.80,Ricky,1,122.47


In [108]:
mwp_df.to_csv(project_dir + '\\data\\matchup_summary\\weekly_scoring_patterns.csv', index=True)